In [19]:
import numpy as np
import math
import random
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

In [16]:
from Arm_Lib import Arm_Device
Arm = Arm_Device()  # Get DOFBOT object

FileNotFoundError: [Errno 2] No such file or directory

In [17]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

Defines Arm related functions for moving the arm. 

In [20]:
def set_angles(arm, angles, t):
    """Sets all the angles of arm
    Args:
        arm - the real robot
        angles - the angles to set
        t - the amount of time to move each angle
    """
    for joint, (angle, movetime) in enumerate(zip(angles, t)):
        arm.Arm_serial_servo_write(joint+1, angle, movetime)

def readAllActualJointAngles(arm):
    q = np.array([arm.Arm_serial_servo_read(id) for id in range(1,6)])
    return q

In [ ]:
import time #import the time module. Used for adding pauses during operation
from Arm_Lib import Arm_Device #import the module associated with the arm

from general_robotics_toolbox import fwdkin, Robot
import general_robotics_toolbox as rox

In [ ]:
# search for gpu
device = torch.device("cuda" if torch.cude.is_available() else "cpu")
print(f'Using {device} (device)")

Define the architechure for the Actor and Critic Networks

In [ ]:
# define the neural networks
"""
The neural network learns an approximation of the value function
with a neural network
"""
class DQN(nn.Module):
    def __init__(self):
        super().__init()
        
    def forward(self, x):
        pass


Initialize Dofbot

In [4]:
ex = np.array([1,0,0])
ey = np.array([0,1,0])
ez = np.array([0,0,1])
l0 = 0.061 # base to servo 1
l1 = 0.0435 # servo 1 to servo 2
l2 = 0.08285 # servo 2 to servo 3
l3 = 0.08285 # servo 3 to servo 4
l4 = 0.07385 # servo 4 to servo 5
l5 = 0.05457 # servo 5 to gripper
P01 = ( l0 + l1 ) * ez 
P12 = np.zeros (3) # translation between 1 and 2 frame in 1 frame
P23 = l2 * ex # translation between 2 and 3 frame in 2 frame
P34 = - l3 * ez # translation between 3 and 4 frame in 3 frame
P45 = np.zeros (3) # translation between 4 and 5 frame in 4 frame
P5T = -( l4 + l5 ) * ex 
print(P01,P12,P23,P34,P45,P5T)

P = np.array([P01, P12, P23, P34, P45, P5T]).T
H = np.array([ez, -ey, -ey, -ey, -ex]).T
print(P,H)
limits = np.array([0,180] * 6).reshape(6, 2)
limits[4, :] = [0, 270]
print(limits)

[0.     0.     0.1045] [0. 0. 0.] [0.08285 0.      0.     ] [-0.      -0.      -0.08285] [0. 0. 0.] [-0.12842 -0.      -0.     ]
[[ 0.       0.       0.08285 -0.       0.      -0.12842]
 [ 0.       0.       0.      -0.       0.      -0.     ]
 [ 0.1045   0.       0.      -0.08285  0.      -0.     ]] [[ 0  0  0  0 -1]
 [ 0 -1 -1 -1  0]
 [ 1  0  0  0  0]]
[[  0 180]
 [  0 180]
 [  0 180]
 [  0 180]
 [  0 270]
 [  0 180]]


In [8]:
from Robot import UR5Arm
dofbot = UR5Arm(P, H, limits)

During the Training Cycle, the following steps are performed for each epoch:
1. Initialize the Robot to a random state.
2. select a random end effector position that is valid and not self colliding
3. The robot can takes 1 action in a continuous action space for each joint:
   - the DDPG network outputs a value between -5 and 5 and the arm is commanded to
     move each servo that angle amount
4. observe the robot move 

In [ ]:
def collision_check(arm: Arm):
    pass

In [ ]:

def train(episodes: int, virtual_arm: UR5Arm, real_arm: Arm):

    ms_per_angle = 6
    # coefficient for time weight
    t_coeff = 0.1
    q_network = DQN().to(device)

    # optimization and ML constants
    for ep in episodes:
        # select target destination for arm
        """
        target_end_effector is used to evaluate the model.
        The farther the euclidean distance from the end effector,
        the lower the reward.
        """
        collision = True
        target_angles = np.zeros(5,1)
        while not collision:
            target_angles = np.random.rand(5, 1) * virtual_arm.get_limits()[:,1].flatten())
            collision = collision_check(arm, target_angles)
        target_end_effector_pose = fwdkin(dofbot, target_angles)

        """
        the state are the current servo
        """
        start_collision = True
        starting_angles = np.zeros(5,1)
        while not start_collision:
            starting_angles = np.random.rand(5, 1) * virtual_arm.get_limits()[:,1].flatten())
            collision = collision_check(arm, starting_angles)
        
        # initialize the arm to a random state. 
        set_angles(real_arm, starting_angles, [1000] * 5)

        # allow the arm to move for 5 seconds
        time_start = time.time()
        elapsed_time = 0
        while(elapsed_time - time_start < 10):
            curr_angles = readAllActualJointAngles()
            current_end_effector_pose = fwdkin(dofbot, curr_angles)
    
            observations = curr_angles, current_end_effector_pose, target_end_effector_pose
            dq = q_network(observations)

            next_angles = curr_angles + dq
            """
            TO DO 
            check the joint limits and 
            penalize if the network tried to go over the joint limits
            """
            set_angles(real_arm, next_angles, dq * ms_per_angle)
            
            loss = np.linalg.norm(target_angles - curr_angles) + tcoef * elapsed_time
            """
            TO DO
            gradient descent on the q network here based on loss
            """          
        
